In [1]:
from tensorflow import keras
import numpy as np

RND_SEED = 12345

def train_generator():
    return keras.preprocessing.image.ImageDataGenerator(
        validation_split=0.25,
        rescale=1./255.,
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
#        rotation_range=90,
    )

def load_train(path):
    return train_generator().flow_from_directory(
        path,
        target_size=(150, 150),
        batch_size=16,
        class_mode='sparse',
        subset='training',
        seed=RND_SEED)

def create_model(input_shape):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(input_shape))

    model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')) #, input_shape=input_shape
    model.add(keras.layers.AvgPool2D(pool_size=(2, 2)))

    model.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='relu'))
    model.add(keras.layers.AvgPool2D(pool_size=(2, 2)))

    model.add(keras.layers.Conv2D(filters=12, kernel_size=(3, 3), padding='valid', activation='relu'))
    model.add(keras.layers.AvgPool2D(pool_size=(2, 2)))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=120, activation='relu'))
    model.add(keras.layers.Dense(units=84, activation='relu'))
    model.add(keras.layers.Dense(units=12, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy', metrics=['acc'])
    return model

def train_model(model, train_data, test_data, batch_size=None, epochs=5,
               steps_per_epoch=None, validation_steps=None):
    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)
    return model

2025-09-17 11:21:55.243147: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-17 11:21:55.666078: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-17 11:21:57.643291: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
image_gen = train_generator()
train_datagen_flow = image_gen.flow_from_directory(
			'./datasets/fruits_small/',
			target_size=(150, 150),
			batch_size=16,
			class_mode='sparse',
			subset='training',
			seed=RND_SEED)
val_datagen_flow = image_gen.flow_from_directory(
			'./datasets/fruits_small/',
			target_size=(150, 150),
			batch_size=16,
			class_mode='sparse',
			subset='validation',
			seed=RND_SEED)

Found 1266 images belonging to 12 classes.
Found 417 images belonging to 12 classes.


In [3]:
features, target = next(train_datagen_flow)
len(train_datagen_flow)

80

In [4]:
print('input shape:', features[0].shape)
model = create_model(features[0].shape)
model = train_model(model,
	train_data=train_datagen_flow,
	test_data=val_datagen_flow)

input shape: (150, 150, 3)


2025-09-17 11:21:58.838254: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/wtom/code/notebooks/practicum/vision/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
80/80 - 19s - 238ms/step - acc: 0.1983 - loss: 2.2412 - val_acc: 0.2614 - val_loss: 1.9735
Epoch 2/5
80/80 - 17s - 217ms/step - acc: 0.2543 - loss: 1.8696 - val_acc: 0.3237 - val_loss: 1.8062
Epoch 3/5
80/80 - 17s - 217ms/step - acc: 0.4250 - loss: 1.5054 - val_acc: 0.5827 - val_loss: 1.1190
Epoch 4/5
80/80 - 17s - 214ms/step - acc: 0.6161 - loss: 1.0387 - val_acc: 0.6715 - val_loss: 0.7740
Epoch 5/5
80/80 - 17s - 214ms/step - acc: 0.7014 - loss: 0.8085 - val_acc: 0.7482 - val_loss: 0.6714


val_acc: 0.8273